In [4]:
import distiller 
import numpy as np
import os, collections
import bitstring 
import time 
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms
import torch.nn.functional as F
import torchvision 
import models 
import matplotlib
from matplotlib import pyplot as plt
from eval_util import test_imagenet 
# import multiprocessing 
%matplotlib inline

from fault_injection import * 

matplotlib.rcParams['ps.useafm'] = True
matplotlib.rcParams['pdf.use14corefonts'] = True
matplotlib.rcParams['text.usetex'] = True

print('using GPU:', torch.cuda.is_available())
torch.manual_seed(1)

using GPU: True


In [7]:
# check the weight distribution of other pre-trained models in torch vision
# use resnet18, alexnet, vgg16 

pretrained_models = {'resnet18': torchvision.models.resnet18(pretrained=True),
                     'resnet34': torchvision.models.resnet34(pretrained=True),
                     'alexnet': torchvision.models.alexnet(pretrained=True),
#                      'squeezenet': torchvision.models.squeezenet1_0(pretrained=True),
                     'vgg16':  torchvision.models.vgg16(pretrained=True), 
                      'vgg16_bn':  torchvision.models.vgg16_bn(pretrained=True), 
#                      'densenet':  torchvision.models.densenet161(pretrained=True),
                     'inception_v3':  torchvision.models.inception_v3(pretrained=True),
                    }
# for model_name, pretrained_model in pretrained_models.items():
#     tensor = [param.data.cpu().numpy().ravel() for param in pretrained_model.parameters()]
#     tensor = np.concatenate(tensor)
#     minv, maxv = np.min(tensor), np.max(tensor)
#     num_values = tensor.shape[0]
#     plt.hist(tensor, bins=10000)
#     plt.title(model_name+ ':[%f, %f], #=%.1f(M)' %(minv, maxv, num_values/10e6))
#     plt.show()

model_name = 'vgg16'
model = pretrained_models[model_name]
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [8]:
# check model weight size is a multiple of eight
for name, param in model.named_parameters():
    if len(param.size()) < 2:
        continue 
    length = param.nelement()
    print(name, param.size(), '8X?', length%8 == 0)
    

features.0.weight torch.Size([64, 3, 3, 3]) 8X? True
features.2.weight torch.Size([64, 64, 3, 3]) 8X? True
features.5.weight torch.Size([128, 64, 3, 3]) 8X? True
features.7.weight torch.Size([128, 128, 3, 3]) 8X? True
features.10.weight torch.Size([256, 128, 3, 3]) 8X? True
features.12.weight torch.Size([256, 256, 3, 3]) 8X? True
features.14.weight torch.Size([256, 256, 3, 3]) 8X? True
features.17.weight torch.Size([512, 256, 3, 3]) 8X? True
features.19.weight torch.Size([512, 512, 3, 3]) 8X? True
features.21.weight torch.Size([512, 512, 3, 3]) 8X? True
features.24.weight torch.Size([512, 512, 3, 3]) 8X? True
features.26.weight torch.Size([512, 512, 3, 3]) 8X? True
features.28.weight torch.Size([512, 512, 3, 3]) 8X? True
classifier.0.weight torch.Size([4096, 25088]) 8X? True
classifier.3.weight torch.Size([4096, 4096]) 8X? True
classifier.6.weight torch.Size([1000, 4096]) 8X? True


In [9]:
valdir = '/home/hguan2/datasets/imagenet/val'
# acc1 = test_imagenet(model, valdir, num_batches=50)

In [10]:
# post train quantization 
quantizer = distiller.quantization.PostTrainLinearQuantizer(model)
quantizer.prepare_model()
# print(model)

In [11]:
acc1 = test_imagenet(model, valdir, num_batches=50)

Test: [  0/500]	Time  1.823 ( 1.823)	Loss 5.4260e-01 (5.4260e-01)	Acc@1  91.00 ( 91.00)	Acc@5  95.00 ( 95.00)
Test: [ 10/500]	Time  0.984 ( 1.155)	Loss 7.2956e-01 (5.3209e-01)	Acc@1  85.00 ( 87.64)	Acc@5  94.00 ( 96.73)
Test: [ 20/500]	Time  0.988 ( 1.143)	Loss 7.2058e-01 (7.1135e-01)	Acc@1  79.00 ( 82.19)	Acc@5  95.00 ( 95.67)
Test: [ 30/500]	Time  1.037 ( 1.119)	Loss 1.6181e+00 (8.3713e-01)	Acc@1  58.00 ( 79.00)	Acc@5  94.00 ( 94.52)
Test: [ 40/500]	Time  0.974 ( 1.109)	Loss 5.5259e-01 (8.8682e-01)	Acc@1  87.00 ( 77.44)	Acc@5  97.00 ( 93.95)
Acc@1: 79.360, Acc@5: 94.340


In [12]:
# use SEC_DED code to encode tensor inplace. use the LSB to record the parity bit
model.cpu() 
for name, param in model.named_parameters():
    if len(param.size()) < 2:
        continue 
    tensor = param.data
    
    start = time.time()
    secded_encode(tensor)
    end = time.time() - start
    print('encode tensor name:%s, size:%s, time(s):%s' %(name, tensor.nelement(), end))
    
acc1 = test_imagenet(model, valdir, num_batches=50)

encode tensor name:features.0.wrapped_module.weight, size:1728, time(s):0.05556011199951172
encode tensor name:features.2.wrapped_module.weight, size:36864, time(s):1.0874993801116943
encode tensor name:features.5.wrapped_module.weight, size:73728, time(s):2.2943947315216064
encode tensor name:features.7.wrapped_module.weight, size:147456, time(s):4.451993942260742
encode tensor name:features.10.wrapped_module.weight, size:294912, time(s):9.058299541473389
encode tensor name:features.12.wrapped_module.weight, size:589824, time(s):18.209731340408325
encode tensor name:features.14.wrapped_module.weight, size:589824, time(s):18.156004190444946
encode tensor name:features.17.wrapped_module.weight, size:1179648, time(s):36.30216360092163
encode tensor name:features.19.wrapped_module.weight, size:2359296, time(s):72.84387922286987
encode tensor name:features.21.wrapped_module.weight, size:2359296, time(s):72.85865712165833
encode tensor name:features.24.wrapped_module.weight, size:2359296, t

In [28]:
def get_named_weights(model):
    named_params = [] 
    for name, param in model.named_parameters():
        if len(param.size()) >= 2:
            named_params.append((name, param)) 
    return named_params 

def large_value_percentage(tensor):
    size = tensor.nelement()
    num_large_values = torch.nonzero((tensor > 63) + (tensor < -64)).size()[0]
#     print(num_large_values)
    return num_large_values*1.0/size 

# def test_large_value_percentage():
#     tensor = torch.randint(-100, 100, size=(10, ))
#     print(tensor)
#     print(large_value_percentage(tensor))
# test_large_value_percentage()
    
named_params = get_named_weights(model)

for name, param in named_params:
    print(name, large_value_percentage(param.data))

features.0.wrapped_module.weight 0.02488425925925926
features.2.wrapped_module.weight 0.0030381944444444445
features.5.wrapped_module.weight 0.0006374782986111111
features.7.wrapped_module.weight 0.0012071397569444445
features.10.wrapped_module.weight 4.0690104166666664e-05
features.12.wrapped_module.weight 4.0690104166666664e-05
features.14.wrapped_module.weight 8.477105034722223e-06
features.17.wrapped_module.weight 2.8822157118055555e-05
features.19.wrapped_module.weight 3.1789143880208336e-05
features.21.wrapped_module.weight 9.1552734375e-05
features.24.wrapped_module.weight 7.205539279513889e-06
features.26.wrapped_module.weight 4.662407769097223e-06
features.28.wrapped_module.weight 0.000705718994140625
classifier.0.wrapped_module.weight 2.452305385044643e-06
classifier.3.wrapped_module.weight 0.002543628215789795
classifier.6.wrapped_module.weight 0.000566650390625


In [7]:
# check the distribution of parameters 
# thr = 32
# layer_id = 0 
# for param_name, param in model.named_parameters():
#     if len(param.size()) < 2:
#         continue
#     counter = collections.Counter(np.abs(param.data.cpu().numpy().ravel())//thr)
#     tmp = sorted(counter.items(), key=lambda x: x[0])
#     values, counts = zip(*tmp)
#     percentages = [count/sum(list(counts)) for count in counts]
#     bar = plt.bar(values, percentages)
#     for rect in bar:
#         height = rect.get_height()
#         plt.text(rect.get_x() + rect.get_width()/2.0, height, '%.4f%%' %(height*100), ha='center', va='bottom')
# #     print(['%.2f' %(p) for p in percentages])
#     #plt.hist(param.data.cpu().numpy().ravel(), bins=10, density=True)
#     plt.xticks(values, [str(int(v)*thr+thr) for v in values])
#     plt.title('layer_id:'+str(layer_id) + ', '+ str(tuple(param.size())))
# #     plt.grid()
#     plt.ylim(0, 1.1)
#     plt.show()
#     layer_id += 1

In [8]:
# check the distribution of parameters all weights
thr = 32
total_values, num_weights = 0, 0 
counter = collections.Counter()
for param_name, param in model.named_parameters():
    total_values += param.nelement()
    if len(param.size()) < 2:
        continue
    num_weights += param.nelement()
    counter.update(collections.Counter(np.abs(param.data.cpu().numpy().ravel())//thr + 1))
    
tmp = sorted(counter.items(), key=lambda x: x[0])
values, counts = zip(*tmp)
total_weights = sum(list(counts))

assert total_weights == num_weights
print('#weights:', total_weights, ', #params:', total_values, 'percentage:', '%.6f' %(num_weights/total_values))

percentages = [count/total_weights for count in counts]
bar = plt.bar(values, percentages)
for rect in bar:
    height = rect.get_height()
    plt.text(rect.get_x() + rect.get_width()/2.0, height, '%.4f%%' %(height*100), ha='center', va='bottom')
#     print(['%.2f' %(p) for p in percentages])
#plt.hist(param.data.cpu().numpy().ravel(), bins=10, density=True)
# plt.xticks(values, [str(int(v)*thr) for v in values])
plt.xticks(values, ['[0, 32]', '[32, 64]', '[64, 96]', '[96, 128]'])
plt.title(model_name)
#     plt.grid()
plt.ylim(0, 1.1)
plt.show()

figdir = './figures/weight_distribution/'
figname = model_name+'_int8_weight_distribution.pdf'
plt.savefig(os.path.join(figdir, figname), bbox_inches='tight')


#weights: 138344128 , #params: 138357544 percentage: 0.999903


FileNotFoundError: [Errno 2] No such file or directory: 'latex'

<Figure size 432x288 with 1 Axes>

NameError: name 'f' is not defined